In [ ]:
#LIBRARIES
# import numpy as np
# import pandas as pd
# import matplotlib.pyplot as plt
# %matplotlib inline
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import accuracy_score
# from mpl_toolkits.mplot3d import Axes3D

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

print("All libraries loaded!")


In [ ]:
#ATTEMPT1
df = 

3.12.2 (main, Feb 10 2024, 20:19:56) [Clang 15.0.0 (clang-1500.1.0.2.5)]
